<a href="https://colab.research.google.com/github/lasteemo/test_project/blob/main/text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import cv2
import numpy as np
from glob import glob
import random
import xml.etree.ElementTree as ET

class preprocessing(object):
    def __init__(self, resize_size):
        self.resize_size = resize_size
    
    def resize(self, img):
        resize_img = cv2.resize(img, (self.resize_size, self.resize_size))
        return resize_img
    
    def normalize_img(self, img):
        norm_img = img / 255.0
        return norm_img
    
    def label_parsing(self, label_path):
        info = ET.parse(label_path)
        root = info.getroot()
        member_object = root.findall('object')
        label_data = []
        labels_dict = dict(obj_class=[], xmin=[], xmax=[], ymin=[], ymax=[])
        label_data.append(label_path)
        for obj in member_object:
            if obj.find('name').text == 'apple':
                obj_class = 0
            elif obj.find('name').text == 'banana':
                obj_class = 1
            elif obj.find('name').text == 'orange':
                obj_class = 2
            
            labels_info = obj.find('bndbox')
            xmin = int(labels_info.find('xmin').text)
            xmax = int(labels_info.find('xmax').text)
            ymin = int(labels_info.find('ymin').text)
            ymax = int(labels_info.find('ymax').text)
            labels_dict['xmin'].append(xmin)
            labels_dict['xmax'].append(xmax)
            labels_dict['ymin'].append(ymin)
            labels_dict['ymax'].append(ymax)
            labels_dict['obj_class'].append(obj_class)
            label_data.append(labels_dict)
        return label_data
def xml_to_txt(path):
    for xml_file in glob.glob(path + '/*.xml'):
        everyrow_xml_list = []
        tree = ET.parse(xml_file)
        root = tree.getroot()
        everyrow_xml_list.append(path + '/' + root.find('filename').text)
        for member in root.findall('object'):
            xmin = str(int(member[4][0].text))
            ymin = str(int(member[4][1].text))
            xmax = str(int(member[4][2].text))
            ymax = str(int(member[4][3].text))
            if xmin=="0":
                xmin="1"
            if ymin=="0":
                ymin="1"
            if xmax=="0":
                xmax="1"
            if ymax=="0":
                ymax="1"
            value = xmin+','+ymin+','+xmax+','+ymax+','+member[0].text
            everyrow_xml_list.append(value)
        txt_list.append(everyrow_xml_list)#image_path x_min,y_min,x_max,y_max,class_id  x_min,y_min,x_max,y_max,class_id ……
    return txt_list




def main():
    datapath = './drive/MyDrive/fruit/test'
    image_files = glob(os.path.join(datapath, '*.jpg'))
    label_files = glob(os.path.join(datapath, '*.xml'))
    prepro_img = preprocessing(resize_size=244)
    
    data = list(zip(image_files, label_files))
    random.seed(4)
    random.shuffle(data)
    
    # check the input images
    for file in data:
        #print(file)
        image = cv2.imread(file[0])
        label = prepro_img.label_parsing(file[1])
        
        #resize_img = prepro_img.resize(image)
        #norm_img = prepro_img.normalize_img(resize_img)
 
        if (file[0].split('\\'))[-1] == 'mixed_8.jpg':
            rect_img = image
            #print(len(label[1]['obj_class']))
            for i in range(len(label[1]['obj_class'])):
                bb_color = (0, 0, 0)
                if label[1]['obj_class'][i] == 0:
                    bb_color = (255, 0, 0) # blue: apple
                elif label[1]['obj_class'][i] == 1:
                    bb_color = (0, 255, 0) # green: banana
                elif label[1]['obj_class'][i] == 2:
                    bb_color = (0, 0, 255) # red: orange
                
                rect_img = cv2.rectangle(rect_img, (label[1]['xmin'][i], label[1]['ymin'][i]), (label[1]['xmax'][i], label[1]['ymax'][i]), bb_color, 1)
            cv2.imwrite('bb_result.jpg', rect_img)
     
    with open('test.txt', 'w') as f:
        for file in data:
            label = prepro_img.label_parsing(file[1])
            f.write(label[0]+' ')
            for i in range(len(label[1]['obj_class'])):
                f.write('{} {} {} {} {}'.format(label[1]['obj_class'][i], label[1]['xmin'][i], label[1]['xmax'][i], label[1]['ymin'][i], label[1]['ymax'][i]))
            f.write('\n')
        
    #train_step

    #valid_step


if __name__=="__main__":
    main()

In [ ]:
import os
import glob
import csv
import xml.etree.ElementTree as ET

os.chdir(r'/content/drive/MyDrive/fruit/train')
path = r'/content/drive/MyDrive/fruit/train'

def xml_to_txt(path):
    txt_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        everyrow_xml_list = []
        tree = ET.parse(xml_file)
        root = tree.getroot()
        everyrow_xml_list.append(path + '/' + root.find('filename').text)
        for member in root.findall('object'):
            xmin = str(int(member[4][0].text))
            ymin = str(int(member[4][1].text))
            xmax = str(int(member[4][2].text))
            ymax = str(int(member[4][3].text))
            if xmin=="0":
                xmin="1"
            if ymin=="0":
                ymin="1"
            if xmax=="0":
                xmax="1"
            if ymax=="0":
                ymax="1"
            if member.find('name').text == 'apple':
              obj_class = "0"
            elif member.find('name').text == 'banana':
              obj_class = "1"
            elif member.find('name').text == 'orange':
              obj_class = "2"
            value = xmin+','+ymin+','+xmax+','+ymax+','+obj_class
            everyrow_xml_list.append(value)
        txt_list.append(everyrow_xml_list)#image_path x_min,y_min,x_max,y_max,class_id  x_min,y_min,x_max,y_max,class_id ……
    return txt_list



def main():
    image_path = path
    xml2txt_list = xml_to_txt(image_path)
    with open(r'/content/drive/MyDrive/fruit/train.txt', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f,delimiter=' ')
        writer.writerows(xml2txt_list)
    print('Successfully converted xml to txt.')

if __name__ == '__main__':
    main()

Successfully converted xml to txt.


In [ ]:
import os
import glob
import csv
import xml.etree.ElementTree as ET

os.chdir(r'/content/drive/MyDrive/fruit/test')
path = r'/content/drive/MyDrive/fruit/test'

def xml_to_txt(path):
    txt_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        everyrow_xml_list = []
        tree = ET.parse(xml_file)
        root = tree.getroot()
        everyrow_xml_list.append(path + '/' + root.find('filename').text)
        for member in root.findall('object'):
            xmin = str(int(member[4][0].text))
            ymin = str(int(member[4][1].text))
            xmax = str(int(member[4][2].text))
            ymax = str(int(member[4][3].text))
            if xmin=="0":
                xmin="1"
            if ymin=="0":
                ymin="1"
            if xmax=="0":
                xmax="1"
            if ymax=="0":
                ymax="1"
            if member.find('name').text == 'apple':
              obj_class = "0"
            elif member.find('name').text == 'banana':
              obj_class = "1"
            elif member.find('name').text == 'orange':
              obj_class = "2"
            value = xmin+','+ymin+','+xmax+','+ymax+','+obj_class
            everyrow_xml_list.append(value)
        txt_list.append(everyrow_xml_list)#image_path x_min,y_min,x_max,y_max,class_id  x_min,y_min,x_max,y_max,class_id ……
    return txt_list



def main():
    image_path = path
    xml2txt_list = xml_to_txt(image_path)
    with open(r'/content/drive/MyDrive/fruit/test.txt', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f,delimiter=' ')
        writer.writerows(xml2txt_list)
    print('Successfully converted xml to txt.')

if __name__ == '__main__':
    main()

Successfully converted xml to txt.
